In [1]:
import os
import json
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Change the current working directory to the pachage root
# That's step is due to the way settings.py is defined
ROOT_DIR = os.path.join(*os.path.split(os.getcwd())[:-1])
os.chdir(ROOT_DIR)
os.getcwd()

'/Users/luisrodrigues/Documents/Projects/PERSONAL/resume-worth'

In [3]:
conf_dir = os.path.join("conf")
conf_file = ".env"

data_dir = os.path.join("data", "01_raw")
data_file = "data_scientist_jobstreet_scraped_v2 2.csv"

res_dir = os.path.join("data", "02_processed")

res_metadata_dir = os.path.join("data", "02_processed", "metadata")

In [4]:
# Load OpenAI API Key
# https://openai.com/pricing

conf_path = os.path.join(conf_dir, conf_file)

_ = load_dotenv(conf_path)

#API_KEY = os.getenv('API_KEY')

In [5]:
data_path = os.path.join(data_dir, data_file)

data_df = pd.read_csv(data_path, sep=",")

In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    606 non-null    int64 
 1   job_id        606 non-null    int64 
 2   job_title     606 non-null    object
 3   company       606 non-null    object
 4   descriptions  588 non-null    object
 5   location      606 non-null    object
 6   category      606 non-null    object
 7   subcategory   606 non-null    object
 8   type          606 non-null    object
 9   salary        198 non-null    object
dtypes: int64(2), object(8)
memory usage: 47.5+ KB


In [7]:
display(data_df.sample(n=5))

,Unnamed: 0,job_id,job_title,company,descriptions,location,category,subcategory,type,salary
381,381,72235414,Data Scientist (Credit),Michael Page International (Malaysia) Sdn Bhd,Lead requirement gathering discussions for use...,Kuala Lumpur,Banking & Financial Services,Analysis & Reporting,Full time,NaN
469,469,72700478,RPA Manager (UiPath),Ambition Group Malaysia Sdn Bhd,Develop and execute a comprehensive UiPath aut...,Kuala Lumpur,Information & Communication Technology,Team Leaders,Full time,"RM 7,000 – RM 10,000 per month"
167,167,72376611,Head of Advanced Data Analytics,Agensi Pekerjaan Hays (Malaysia) Sdn Bhd,Being the point-of-contact for advanced analyt...,Kuala Lumpur,Science & Technology,"Mathematics, Statistics & Information Sciences",Full time,"RM 20,000 – RM 30,000 per month"
314,314,72750107,IT Business Analyst,Mewah Group,To develop new software applications and custo...,Klang District,Information & Communication Technology,Business/Systems Analysts,Full time,NaN
5,5,72762762,Senior Engineer / Engineer - Data Management &...,B. Braun Medical Industries Sdn. Bhd.,"Build, maintain and utilize a variety of datab...",Penang,Engineering,Other,Full time,NaN


In [8]:
data_clean_df = data_df[['job_id', 'location', 'company', 'job_title', 'salary', 'descriptions']].dropna().drop_duplicates().copy()

In [9]:
len(data_clean_df)

32

In [10]:
data_clean_df.groupby('job_title')[['job_id']].count().reset_index().sort_values(by="job_id",ascending=False)

,job_title,job_id
4,Data Analyst,5
7,Data Engineer,4
0,BI Developer - Immediate Hiring,1
14,Intern - Data Science,1
23,System Analyst,1
22,Scientist (Bioinformatics),1
21,RPA Manager (UiPath),1
20,"Principal Engineer, Automation Project",1
19,MicroStrategy Developers (MNC),1
18,"Manager, RPA Developer",1


In [11]:
for index, row in data_clean_df.iterrows():

    job_vacancy = {
        "id": row['job_id'],
        "location": row['location'].lower(),
        "company": row['company'].lower(),
        "job_title": row['job_title'].lower(),
        "salary": row['salary'],
        "description": row['descriptions'],
    }

    res_file = "{job_id}.json".format(job_id=row['job_id'])
    res_path = os.path.join(res_dir, res_file)

    with open(res_path, "w") as f:
            json.dump(job_vacancy, f)

In [12]:
JOB_TITLES = ["data engineer", "data scientist", "data analyst", "machine learning engineer"]

scraped_job_titles = data_clean_df['job_title'].values

job_vacancy_metadata = {}
for job_title in JOB_TITLES:
    related_job_vacancies = set([job.lower() for job in scraped_job_titles if job_title.lower() in job.lower()])
    job_vacancy_metadata[job_title] = list(related_job_vacancies)

job_vacancy_metadata

{'data engineer': ['data analysts/ data engineers (bangsar south- kerinchi lrt)',
  'data engineer'],
 'data scientist': [],
 'data analyst': ['data analysts/ data engineers (bangsar south- kerinchi lrt)',
  'junior data analyst',
  'data analyst executive',
  'e-commerce data analyst',
  'data analyst'],
 'machine learning engineer': ['machine learning engineer (artificial intelligence)',
  'machine learning engineer (ai)']}

In [14]:
try:
    os.mkdir(res_metadata_dir)
except:
    pass

res_file = "job_vacancy_metadata.json"
res_path = os.path.join(res_metadata_dir, res_file)

with open(res_path, "w") as f:
        json.dump(job_vacancy_metadata, f)